In [23]:
!pip install ortools

In [39]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from scipy.spatial import distance
# Importing the geodesic module from the library 
from geopy.distance import geodesic 

from statistics import mean

import time
import logging

import pandas as pd
#To plot maps of the solution
import folium 



coords=pd.read_csv("TSP.csv")
# coords = coord[:-3]
print("Points to consider for optimization are:")
#Variable where the start node or "Home" is entered, this MUST be an integer value between 0 - len(coords), the user should select this
PUNTO_DE_INICIO=4 

#This block assigns lat, long to separate lists
array=coords.values.tolist()
latitudes =[]
longitudes=[]
for i in range (0,len(array)):
    latitudes.append(array[i][0])
    longitudes.append(array[i][1])

basemap = folium.Map(
            location=[mean(latitudes), mean(longitudes)],
            zoom_start=11)


#This block plots the points to be considered
tooltip = 'Informacion'
for i in range (0,len(array)):
    corresponding_icon='truck'
    corresponding_color='blue'
    
    if i==PUNTO_DE_INICIO:
        corresponding_icon='home'
        corresponding_color='green'
    
    folium.Marker(
    location=[latitudes[i], longitudes[i]],
    tooltip=tooltip,
    popup=('Punto: '+str(i)),
    icon=folium.Icon(icon=corresponding_icon, prefix='fa',color=corresponding_color)
    ).add_to(basemap)
basemap

Points to consider for optimization are:


In [40]:
coords

,latitude,longitude
0,18.93980,72.83540
1,19.13510,72.81460
2,19.05960,72.82950
3,19.11760,72.90600
4,19.08789,72.89224
5,19.01780,72.84780
6,19.02220,72.92690
7,19.11420,72.93580
8,19.11360,72.86970
9,19.00075,72.81237


In [25]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)
logger.warning('LOGGER OK & READY')

LOGGER OK & READY
LOGGER OK & READY
LOGGER OK & READY


In [26]:
start_time = time.time()

print("The distance matrix (geodesic distance) is:\n")
array=coords.values.tolist()
distance_matrix=[]
for i in range(0,len(array)):
    inner_list=[]
    for j in range (0,len(array)):
        orig=tuple(array[i])
        dest=tuple(array[j])
        distancia_i=round(geodesic(orig, dest).m) 
        inner_list.append(distancia_i)
    distance_matrix.append(inner_list)
print(distance_matrix)
end_time=(time.time() - start_time)
logger.warning("This process took :"+str(end_time/60)+" minutes to complete.")

This process took :0.0005355000495910645 minutes to complete.
This process took :0.0005355000495910645 minutes to complete.
This process took :0.0005355000495910645 minutes to complete.


The distance matrix (geodesic distance) is:

[[0, 21729, 13275, 21038, 17451, 8732, 13268, 22009, 19574, 7169], [21729, 0, 8503, 9810, 9698, 13446, 17201, 12960, 6267, 14873], [13275, 8503, 0, 10297, 7308, 5012, 11057, 12715, 7323, 6759], [21038, 9810, 10297, 0, 3593, 12632, 10787, 3158, 3845, 16261], [17451, 9698, 7308, 3593, 0, 9059, 8135, 5431, 3705, 12796], [8732, 13446, 5012, 12632, 9059, 0, 8342, 14130, 10852, 4181], [13268, 17201, 11057, 10787, 8135, 8342, 0, 10227, 11773, 12290], [22009, 12960, 12715, 3158, 5431, 14130, 10227, 0, 6955, 18069], [19574, 6267, 7323, 3845, 3705, 10852, 11773, 6955, 0, 13873], [7169, 14873, 6759, 16261, 12796, 4181, 12290, 18069, 13873, 0]]


In [31]:
start_time = time.time()


distancia_maxima=0
for i in range(0,len(distance_matrix)):
    for j in range(0,len(distance_matrix[i])):
        if distance_matrix[i][j]>distancia_maxima:
            distancia_maxima=distance_matrix[i][j]
            
distancia_maxima 

"""Input Parameters for the model """

num_vehicles=3 
vehicle_maximum_travel_distance=(len(distance_matrix)*distancia_maxima) 
#vehicle_maximum_travel_distance=60000 
depot=PUNTO_DE_INICIO 

"""Vehicles Routing Problem (VRP)."""

SOLUTION_DATAFRAME=pd.DataFrame() 
SOLUTION_DATAFRAME['Index']=coords.index
#SOLUTION_DATAFRAME_1=pd.DataFrame()
#SOLUTION_DATAFRAME_2=pd.DataFrame()

SOLUTION_LIST=[]
SOLUTION_LIST_2=[]
CONTADOR_DF_SOLUCION=0

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix 
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        #IBP
        SOLUTION_LIST=[] 
        #IBP
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            #IBP
            SOLUTION_LIST.append(manager.IndexToNode(index))
            SOLUTION_LIST_2=pd.Series(SOLUTION_LIST)
            #IBP
            #print(manager.IndexToNode(index)) #IBP
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        #if CONTADOR_DF_SOLUCION ==0:
        SOLUTION_DATAFRAME['Route for Vehicle: '+str(vehicle_id)]=SOLUTION_LIST_2 
          #  SOLUTION_DATAFRAME_2=pd.concat([SOLUTION_DATAFRAME_0, SOLUTION_DATAFRAME_2], axis=1) 
        #else:
         #   SOLUTION_DATAFRAME_1['Route for Vehicle: '+str(vehicle_id)]=SOLUTION_LIST_2
          #  SOLUTION_DATAFRAME_2=pd.concat([SOLUTION_DATAFRAME_2, SOLUTION_DATAFRAME_1], axis=1)
    print('Maximum of the route distances: {}m'.format(max_route_distance))



def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        vehicle_maximum_travel_distance,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('NO SOLUTION')


if __name__ == '__main__':
    main()

end_time=(time.time() - start_time)
logger.warning("This process took :"+str(end_time)+" seconds to complete.")

This process took :0.08519840240478516 seconds to complete.
This process took :0.08519840240478516 seconds to complete.
This process took :0.08519840240478516 seconds to complete.


Route for vehicle 0:
 4 ->  8 ->  1 ->  3 ->  7 -> 4
Distance of the route: 28371m

Route for vehicle 1:
 4 ->  0 -> 4
Distance of the route: 34902m

Route for vehicle 2:
 4 ->  2 ->  9 ->  5 ->  6 -> 4
Distance of the route: 34725m

Maximum of the route distances: 34902m


In [36]:
distance_matrix

[[0, 21729, 13275, 21038, 17451, 8732, 13268, 22009, 19574, 7169],
 [21729, 0, 8503, 9810, 9698, 13446, 17201, 12960, 6267, 14873],
 [13275, 8503, 0, 10297, 7308, 5012, 11057, 12715, 7323, 6759],
 [21038, 9810, 10297, 0, 3593, 12632, 10787, 3158, 3845, 16261],
 [17451, 9698, 7308, 3593, 0, 9059, 8135, 5431, 3705, 12796],
 [8732, 13446, 5012, 12632, 9059, 0, 8342, 14130, 10852, 4181],
 [13268, 17201, 11057, 10787, 8135, 8342, 0, 10227, 11773, 12290],
 [22009, 12960, 12715, 3158, 5431, 14130, 10227, 0, 6955, 18069],
 [19574, 6267, 7323, 3845, 3705, 10852, 11773, 6955, 0, 13873],
 [7169, 14873, 6759, 16261, 12796, 4181, 12290, 18069, 13873, 0]]

In [32]:

solution_map = folium.Map(
            location=[mean(latitudes), mean(longitudes)],
            zoom_start=11.5)

icon_color=['red', 'blue', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray', 'green']

for i in range(0,(len(SOLUTION_DATAFRAME.columns)-1)):
    current_route=pd.merge(SOLUTION_DATAFRAME,coords,left_on=('Route for Vehicle: '+str(i)),right_on=coords.index)
    current_route

    ordered_sol_list=current_route[['latitude','longitude']].values.tolist()
    plot_sol_list=[]
    for j in range (0,len(ordered_sol_list)):
        coord=tuple(ordered_sol_list[j])
        plot_sol_list.append(coord)

        #points=points_tuple_list
        points=plot_sol_list
        k=0
        for each in points:
    
            corresponding_icon='truck'
            corresponding_color=icon_color[i]
    
            if k==0:
                corresponding_icon='home'
                corresponding_color='green'
    
            folium.Marker(
            each,
            tooltip=tooltip,
            popup=('Punto: '+str(int(SOLUTION_DATAFRAME['Route for Vehicle: '+str(i)][k]))),
            icon=folium.Icon(icon=corresponding_icon, prefix='fa',color=corresponding_color)
            ).add_to(solution_map)
            k+=1
  
            folium.PolyLine(points, color=icon_color[i], weight=2.5, opacity=1).add_to(solution_map)
#solution_map

In [33]:
basemap

In [35]:
solution_map